In [9]:
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import pandas as pd

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

In [10]:
data = pd.read_csv("/home/tuandinh/Desktop/Demo/data_final.csv")
data.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 154,Unnamed: 155,Unnamed: 156,Unnamed: 157,Unnamed: 158,Unnamed: 159,Unnamed: 160,Unnamed: 161,Unnamed: 162,label
0,-57.643410,-69.255959,-69.413849,-68.149704,-66.578133,-64.643143,-62.542328,-63.937115,-68.589905,-72.532310,...,-93.558609,-93.287292,-92.827217,-93.226494,-93.012207,-92.636810,-91.491699,-91.801826,-92.639168,0
1,-61.877784,-74.343368,-74.512856,-73.155850,-71.468834,-69.391703,-67.136567,-68.633812,-73.628387,-77.860393,...,-100.431244,-100.139996,-99.646125,-100.074732,-99.844704,-99.441731,-98.212502,-98.545411,-99.444262,0
2,-57.906037,-69.571493,-69.730102,-68.460198,-66.881467,-64.937661,-62.827274,-64.228416,-68.902405,-72.862771,...,-93.984867,-93.712314,-93.250143,-93.651239,-93.435976,-93.058869,-91.908540,-92.220080,-93.061237,0
3,-57.361297,-68.917013,-69.074130,-67.816172,-66.252293,-64.326773,-62.236239,-63.624200,-68.254219,-72.177329,...,-93.100723,-92.830734,-92.372911,-92.770234,-92.556996,-92.183436,-91.043929,-91.352538,-92.185782,0
4,-53.705822,-64.525125,-64.672229,-63.494437,-62.030219,-60.227407,-58.270098,-59.569608,-63.904568,-67.577670,...,-87.167675,-86.914891,-86.486244,-86.858246,-86.658597,-86.308843,-85.241954,-85.530897,-86.311040,0


In [11]:
class SignalDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        """
        Return the total number of samples in the dataset.
        """
        return len(self.data_frame)

    def __getitem__(self, index):
        sample = self.data_frame.iloc[index, :].values.astype(float)

        # The last column can be the target label (0 for no defect, 1 for defect)
        # You can adjust this based on your actual data structure
        target = int(sample[-1])

        # Remove the target label from the sample
        sample = sample[:-1]

        # Convert to PyTorch tensors
        sample = torch.FloatTensor(sample)
        # print(f"Length of sample {len(sample)}")
        target = torch.tensor(target, dtype=torch.long)

        # Apply the optional data transformation
        if self.transform:
            sample = self.transform(sample)

        return sample, target

# Path to your merged CSV file
# csv_merged = '/home/tuandinh/Desktop/Signal Classification/data_signal.csv'
csv_merged = '/home/tuandinh/Desktop/Demo/data_final.csv'
# Create an instance of the custom dataset
custom_dataset = SignalDataset(csv_merged)
# test_dataset = SignalDataset(csv_test_path)

# # Create a data loader for the dataset
# batch_size = 32
# train_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=True)


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# Define a perceptron 
class SimpleClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleClassifier, self).__init__()
        self.fc1 = nn.Linear(163, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 2)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)   
        x = self.sigmoid(x)
        return x
    

In [13]:

# Hyperparameters
input_size = 163  # Number of features in your data
hidden_size = 64  # Number of hidden units in the neural network
num_classes = 2  # Number of classes (0 for no defect, 1 for defect)
learning_rate = 0.000001
num_epochs = 100

# Split your dataset into training and validation sets
train_dataset, val_dataset = train_test_split(custom_dataset, train_size=0.8,  test_size=0.2, random_state=42, shuffle=True)

# Create data loaders for training and validation
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

# Initialize the model, loss function, and optimizer
model = SimpleClassifier(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.to(device=device)


SimpleClassifier(
  (fc1): Linear(in_features=163, out_features=32, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=32, out_features=2, bias=True)
  (sigmoid): Sigmoid()
)

In [14]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.0
    lass_loss = 0

    for i, data in enumerate(train_loader):
        input, labels = data
        optimizer.zero_grad()
        outputs = model(input)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(train_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.
    return lass_loss

In [15]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/singnal_classification_trainer_{}'.format(timestamp))
epoch_number = 0
best_vloss = 1_000_000.

for n in range(num_epochs):
    print('EPOCH {}:'.format(epoch_number + 1))
    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)
    # validation loss
    running_vloss = 0.0
    model.eval()
    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(val_loader):
            val_input, vlabels = vdata
            voutputs = model(val_input)
            vloss = criterion(voutputs, vlabels)
            running_vloss += vloss
        avg_vloss = running_vloss / (i + 1)
        print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))
    
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()
    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'models/model_{}_{}.pt'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

EPOCH 1:
LOSS train 0 valid 0.5729912519454956
EPOCH 2:
LOSS train 0 valid 0.5760024785995483
EPOCH 3:
LOSS train 0 valid 0.5651397109031677
EPOCH 4:
LOSS train 0 valid 0.5565993189811707
EPOCH 5:
LOSS train 0 valid 0.5346755981445312
EPOCH 6:
LOSS train 0 valid 0.5291065573692322
EPOCH 7:
LOSS train 0 valid 0.5236575603485107
EPOCH 8:
LOSS train 0 valid 0.5328245759010315
EPOCH 9:
LOSS train 0 valid 0.5294938087463379
EPOCH 10:
LOSS train 0 valid 0.5257853865623474
EPOCH 11:
LOSS train 0 valid 0.5089524984359741
EPOCH 12:
LOSS train 0 valid 0.5064400434494019
EPOCH 13:
LOSS train 0 valid 0.5186223983764648
EPOCH 14:
LOSS train 0 valid 0.5168060660362244
EPOCH 15:
LOSS train 0 valid 0.515264093875885
EPOCH 16:
LOSS train 0 valid 0.5138176679611206
EPOCH 17:
LOSS train 0 valid 0.5126318335533142
EPOCH 18:
LOSS train 0 valid 0.5115668773651123
EPOCH 19:
LOSS train 0 valid 0.49656155705451965
EPOCH 20:
LOSS train 0 valid 0.5097765326499939
EPOCH 21:
LOSS train 0 valid 0.4948917031288147
E

In [16]:
# data = pd.read_csv("/home/tuandinh/Desktop/Signal Classification/file_data_new.csv")
# # data.shape
# sample = data.iloc[1,:]
# input = sample[:-1]
# label = sample[-1]
# input = torch.tensor(input)
# input.shape

# PATH = "/home/tuandinh/Desktop/Signal Classification/models/model_20230910_093359_0.pt"
# model1 = SimpleClassifier(163, 32,2)
# checkpoint = torch.load(PATH)
# model.load_state_dict(checkpoint)
# model.to(device)
# model.eval()
# # Thực hiện dự đoán
# with torch.no_grad():
#     predicted = model(input)

FileNotFoundError: [Errno 2] No such file or directory: '/home/tuandinh/Desktop/Signal Classification/file_data_new.csv'